In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('sample_data/activity_pp.csv', sep=';', header=None,names=['id', 'activite'])

In [ ]:
data.head(10)

,id,activite
0,A202200592088,"carrosserie, peinture."
1,A202200592210,non précisée
2,A202200592220,non précisée
3,A202200592273,élevage de bovins
4,A202200592474,agent immobilier.
5,A202200602106,transports routiers de marchandises loueur de ...
6,A202200602390,travaux d'installation électrique
7,A202200602728,plomberie chauffage énergie renouvelable
8,A202200612218,"chef d’exploitation, agriculteur"
9,A202200612228,Autres activités récréatives et de loisirs


In [ ]:
#!pip install unidecode

Les secteurs d’activité seront définis à partir d’un échantillon.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

df=data.copy()
# Nettoyage de base
df['activite'] = df['activite'].astype(str)

# Nettoyage NLP

from unidecode import unidecode

def clean_text(text):
    text = text.lower()
    text = unidecode(text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

df['activite_clean'] = df['activite'].apply(clean_text)

# TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['activite_clean'])

# Clustering
kmeans = KMeans(n_clusters=7, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

# Afficher cluster
for i in range(10):
    print(f"\n--- Cluster {i} ---")
    print(df[df['cluster'] == i]['activite'].head(10).to_string(index=False))



--- Cluster 0 ---
Travaux de terrassement courants et travaux pré...
                 travaux d'installation électrique
            travaux de revêtements de sols et murs
                    Travaux de peinture intérieure
                   Travaux de peinture et vitrerie
travaux de pavage, dallage, terrassement, bordu...
maçonnerie, carrelage, ravalement, tous travaux...
Travaux d'installation d'eau et de gaz en tous ...
                 Travaux de menuiserie bois et PVC
    Travaux de menuiserie métallique et serrurerie

--- Cluster 1 ---
                                       immobilière
                            carrosserie, peinture.
                       Restauration traditionnelle
                     Contrôle technique automobile
boulangerie, pâtisserie, Plâts à emporter, comm...
                               Restauration rapide
                       Restauration traditionnelle
                               formateur transport
                                      restau

L'étape précédente m'a aidé à repérer des mots-clés pertinents pour la catégorisation des secteurs.

In [ ]:
# Nettoyage du texte
def nettoyer_texte(texte):
    texte = str(texte).lower()
    texte = re.sub(r'[^a-zàâçéèêëîïôûùüÿñæœ\s]', '', texte)
    return texte

# Classification par mots-clés
def classer_activite(texte):
    texte = nettoyer_texte(texte)
    if any(mot in texte for mot in ['quipement','commerce','commerce de', 'boutique', 'detail',  'epicerie', 'commercial','achat','vente']):
        return 'Commerce & Distribution'
    if any(mot in texte for mot in ['onstruction','ravaux','installat','bâtiment','maçonnerie', 'terrassement', 'peinture', 'menuiserie', 'carrelage', 'btp', 'plomberie','électric']):
        return 'BTP'
    if any(mot in texte for mot in ['transport', 'fret', 'logistique', 'routier', 'camion', 'véhicule', 'location']):
        return 'Transport & Logistique'
    if any(mot in texte for mot in ['bar', 'boisson', 'café']):
        return 'Bar / Café'
    if any(mot in texte for mot in ['restaur', 'traiteur','boulang']):
        return 'Restauration & Hôtellerie'
    if any(mot in texte for mot in ['elevage', 'porcins', 'bovins', 'abattage', 'viande','agriculture', 'fruits', 'legumes', 'champ', 'culture']):
        return 'Élevage /Agriculture/Agroalimentaire'
    if any(mot in texte for mot in ['auto', 'voiture', 'automobile', 'carrosserie', 'vehicule']):
        return 'Services Auto'
    if any(mot in texte for mot in ['esthetique', 'coiffure', 'beaut', 'onglerie']):
        return 'Bien-être / Esthétique'
    if any(mot in texte for mot in ['immobili', 'location', 'logement','hébergement']):
        return 'Immobilier'
    if any(mot in texte for mot in ['informatique', 'Conseil', 'étude','conception','ervice','fournitur', 'fabricat']):
        return 'Service'
    if texte in ['non precisee', 'non precisée', 'non spécifiée','non précisée']:
        return 'Non précisé'

    return 'Autres'

# Appliquer la classification
data['secteur'] = data['activite'].apply(classer_activite)



Cette étape a pour objectif de s'assurer de la cohérence entre les activités identifiées et leur attribution sectorielle, validant ainsi la qualité de la classification.

In [ ]:
data[data['secteur'] == 'Service']


rows_with_informa = data[data['activite'].str.contains('programm', case=False, na=False)]

rows_with_informa

,id,activite,secteur
16,A202200592268,Fabrication d'abris de piscine.,Service
21,A202200592351,Courtage en opérations de banque et en service...,Service
29,A202200592400,conception de produits et services funéraires.,Service
47,A202200602082,Services funéraires,Service
51,A202200602163,Programmation informatique,Service
...,...,...,...
9715,A202401273714,Conseil informatique,Service
9718,A202401273748,"L'étude, la recherche et développement de proc...",Service
9767,A202401282486,"Appareils, articles et fournitures d'optique, ...",Service
9777,A202401282638,Programmation informatique,Service


In [ ]:

# Sauvegarde du résultat dans un nouveau fichier
data.to_csv('sample_data/activities_classe.csv', sep=';', index=False,encoding='utf-8-sig')